In [9]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
from pathlib import Path
from langchain_community.vectorstores import Chroma
# 載入文檔
loader = TextLoader(file_path="books/智慧型手機使用手冊.txt")
documents = loader.load()

# 分割文檔
# chunk_size=1000: 每個文本區塊最多1000字元, 避免超過模型限制
# chunk_overlap=200: 區塊之間重疊200個字元,防止重要資訊在分割時被切斷
text_spliter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_spliter.split_documents(documents)

# 使用 HuggingFace 的中文 embedding 模型
embeddings = HuggingFaceEmbeddings(
    model_name = 'jinaai/jina-embeddings-v2-base-zh'
)

db_path = os.path.abspath("./db/mobile")
if Path(db_path).exists():
    # 資料庫已存在,直接載入
    db = Chroma(persist_directory=db_path, embedding_function=embeddings)
    print("載入現有資料庫") 
else:
    # 資料庫不存在,建立新的向量資料庫
    db = Chroma.from_documents(docs, embeddings, persist_directory=db_path)
    print("建立新資料庫")

results = db.similarity_search("相機功能",k=1)
for result in results:
    print(result)
    print("=" * 60)

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-zh and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.intermedi

載入現有資料庫
page_content='智慧型手機使用手冊

歡迎使用您的新智慧型手機!本手冊將協助您快速上手並充分利用手機的各項功能。

第一章：開始使用

1.1 開箱與首次設定
打開包裝盒後,您會看到以下配件:
- 智慧型手機主機 x1
- USB-C 充電線 x1
- 快速充電器(20W) x1
- SIM 卡退卡針 x1
- 使用說明書 x1
- 保固卡 x1

首次開機設定步驟:
1. 長按電源鍵 3 秒開機
2. 選擇語言(繁體中文)
3. 連接 Wi-Fi 網路
4. 登入或建立 Google 帳號(Android)或 Apple ID(iOS)
5. 設定螢幕鎖定方式(指紋/臉部辨識/密碼)
6. 同意服務條款與隱私權政策
7. 完成設定並進入主畫面

1.2 SIM 卡安裝
1. 找到手機側邊的 SIM 卡槽
2. 使用退卡針插入卡槽旁的小孔
3. 輕輕推入直到卡槽彈出
4. 將 SIM 卡放入卡槽(注意缺角方向)
5. 將卡槽推回手機
6. 等待 5-10 秒識別 SIM 卡

支援的 SIM 卡類型:
- Nano-SIM(4FF)標準
- 雙卡雙待功能
- 5G/4G LTE 網路

第二章:基本操作

2.1 螢幕手勢
- 點擊:輕觸一次開啟應用程式
- 長按:顯示更多選項或移動圖示
- 滑動:切換畫面或捲動內容
- 雙指縮放:放大或縮小圖片/網頁
- 從上往下滑:開啟通知中心
- 從下往上滑:顯示常用功能快捷鍵

2.2 主畫面配置
- 狀態列:顯示時間、電量、訊號強度
- 通知圖示區:顯示未讀訊息、來電等
- 應用程式圖示:點擊開啟應用程式
- 底部導航列:返回、首頁、多工

自訂主畫面:
1. 長按桌面空白處
2. 選擇「新增小工具」或「桌布」
3. 拖曳應用程式圖示來重新排列
4. 建立資料夾:將一個 App 拖到另一個 App 上

2.3 鎖定螢幕設定
支援的解鎖方式:
- 指紋辨識(螢幕下指紋感應)
- 臉部辨識(前置相機)
- 圖形密碼
- PIN 碼(4-6 位數字)
- 密碼(英數混合)

設定解鎖方式:
設定 > 安全性與隱私 > 螢幕鎖定 > 選擇解鎖方式

第三章:相機功能

3.1 相機基本操作
後置主鏡頭規格:
- 5000 萬畫素主鏡頭(f/1.8)
- 1200 萬畫素超廣角鏡頭(120°)
- 80